In [ ]:
''' O projeto se divide em análise de timesires e uma tentativa de prever evazão
fazendo uso de um algoritmo supervisionado
----Passos Time series
1 - Baixar o Log no moodle de eventos no moodle para atividades de participação -
OK
2 - Filtrar os usuário que não são alunos - Excel -pré processamento
OK
3 - Fazer discretização(IP) - Coluna Rede_Puc
OK
4 - Adicionar uma coluna referente ao cronograma(unidade 1 e etc)
Preparando... To do -> Verificar no cronograma 
5 - Adicionar coluna com as turmas
OK
6-  Plotar gráficos com a visualização
------
------ Passos Forecasting
1 - Converter a tabela de time series em um data set anterior em um transicional
    1.1 - Plotar gráfico com estatśticas gerais de uso
    1.2 - Talvez clusterizar os alunos
2 - Descobrir quais usuários trancaram a disciplina - Log de atividades CLI
3 - Descobrir se existe uma data com pico de trancamento
4 - Baseado na relação entre data e trancamento definir um intervalo e filtrar o dataset antes desse pico.
Fazendo isso imagino que posso descobrir algum padrão nos usuários que trancam. Esse será meu target Y
5 - Rodar um random forest
6 - avaliar desempenho
'''
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from datetime import datetime        
%matplotlib inline
sns.set()

In [ ]:
def rede_Puc(turmas):
    ip = '139.82'
    rede = []
    for x in turmas.IP:
        if(ip == x[0:6]):
            rede.append('S')
        else:
            rede.append('N')
    turmas.insert(1, column='Rede_Puc', value=rede)#Passo 5
    return turmas

In [ ]:
def obter_turmas():
    df_ = pd.DataFrame()
    frames = [df_]
    
    for dados_turma in glob.glob('dados/turmas/*'):
        grupo = dados_turma[13:-4]# obtem código do grupo
        df = pd.read_csv(dados_turma)#lê arquivo
        df.insert(3, column='Turma', value=grupo)#insere coluna
        print(df.shape)
        frames.append(df)
    dados = pd.concat(frames)
    dados.columns = [c.replace(' ', '_') for c in dados.columns]#Troca espaço por underline no cabeçalho
    #dados.drop_duplicates(keep=False, inplace=True)#limpando dados duplicados
    dados['Data'], dados['Hora'] = dados['Hora'].str.split(' ', 1).str#divide a coluna Hora em duas colunas
    dados.Data = pd.to_datetime(dados.Data,dayfirst=True)
    
    return dados



In [ ]:
def obter_unidades(t): 
    unidades= []
    data_unidades = {
        'IU1':datetime(2018, 3, 19),
        'FU1':datetime(2018, 4, 18),
        'IU2':datetime(2018, 4, 18),
        'FU2':datetime(2018, 5, 9),
        'IU3':datetime(2018, 5, 9),
        'FU3':datetime(2018, 5, 23),
        'IU4':datetime(2018, 5, 23),
        'FU4':datetime(2018, 6, 27)
    }

    for x in t.Data:
        if(data_unidades['IU1'] <= x < data_unidades['FU1']):
            unidades.append('unidade_1')
        elif(data_unidades['IU2'] <= x < data_unidades['FU2']):
            unidades.append('unidade_2')
        elif(data_unidades['IU3'] <= x < data_unidades['FU3']):
            unidades.append('unidade_3')
        elif(data_unidades['IU4'] <= x < data_unidades['FU4']):
            unidades.append('unidade_4')
        else:
            unidades.append('fora_de_epoca')
            
        
    t.insert(1, column='Unidade', value=unidades)
    return t

   

In [ ]:
turmas = obter_turmas() 
turmas = rede_Puc(turmas)
turmas = obter_unidades(turmas)
turmas

In [ ]:
turmas.to_csv('turmas.csv')

In [ ]:
turmas2=turmas.groupby('Unidade')['Contexto_do_Evento'].value_counts().unstack().fillna(0)#tranformando timesires em data frame
turmas2.to_csv('unidades.csv')